In [1]:
from pykrx import stock
import pandas as pd
from datetime import datetime, timedelta
def get_daily_returns(ticker, start_date="2022-01-01", end_date=None, save_csv=True):
    """
    한국 주식 티커의 일일 수익률을 계산하여 DataFrame 및 CSV로 저장합니다.
    Parameters:
    - ticker: str, 예: "005930" (삼성전자)
    - start_date: str, "YYYY-MM-DD"
    - end_date: str, "YYYY-MM-DD" (기본: 오늘)
    - save_csv: bool, True면 CSV로 저장
    Returns:
    - DataFrame: 날짜별 수익률 포함
    """
    if end_date is None:
        end_date = datetime.today().strftime("%Y-%m-%d")
    # 1. 일별 OHLCV 데이터 가져오기
    df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
    # 2. 수익률 계산 (전일 대비 수익률)
    df['Return'] = df['종가'].pct_change()
    # 3. 날짜 인덱스 리셋
    df.reset_index(inplace=True)
    # 4. 필요한 컬럼만 정리
    df = df[['날짜', '종가', 'Return']]
    # 5. 저장
    if save_csv:
        filename = f"{ticker}_daily_returns.csv"
        df.to_csv(filename, index=False)
        print(f"[✓] Saved to {filename}")
    return df

In [3]:
get_daily_returns('005930', "2025-04-30","2025-06-22")

[✓] Saved to 005930_daily_returns.csv


,날짜,종가,Return
0,2025-04-30,55500,NaN
1,2025-05-02,54300,-0.021622
2,2025-05-07,54600,0.005525
3,2025-05-08,54600,0.000000
4,2025-05-09,54800,0.003663
5,2025-05-12,57600,0.051095
6,2025-05-13,56900,-0.012153
7,2025-05-14,57400,0.008787
8,2025-05-15,57300,-0.001742
9,2025-05-16,56800,-0.008726


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
def lagged_correlation_analysis(df_returns, google_trend, max_lag=7):
    """
    Trend와 미래 수익률 간의 시차 상관관계를 계산합니다.
    Parameters:
    - df_returns: '날짜', 'Return' 컬럼이 있는 DataFrame
    - google_trend: '날짜', 'Trend' 컬럼이 있는 DataFrame
    - max_lag: 최대 시차 (default = 7)
    Returns:
    - lag_corr_df: 시차별 상관계수 DataFrame
    - 가장 높은 상관계수를 가진 시차 정보
    """
    df_returns['날짜'] = pd.to_datetime(df_returns['날짜'])
    google_trend['날짜'] = pd.to_datetime(google_trend['날짜'])
    lag_results = []
    for lag in range(1, max_lag + 1):
        # 트렌드 데이터 그대로 두고, 수익률을 +lag만큼 이동 (즉, 미래 수익률)
        temp_returns = df_returns.copy()
        temp_returns['Return'] = temp_returns['Return'].shift(-lag)  # 미래 수익률로 시프트
        # 날짜 기준 병합
        df_merged = pd.merge(temp_returns, google_trend, on='날짜', how='inner')
        df_clean = df_merged.dropna(subset=['Return', 'Trend'])
        # 상관계수 계산
        corr = df_clean['Return'].corr(df_clean['Trend'])
        lag_results.append({'Lag': lag, 'Correlation': corr})
    # 결과 정리
    lag_corr_df = pd.DataFrame(lag_results)
    # 가장 높은 상관계수 찾기
    best_row = lag_corr_df.loc[lag_corr_df['Correlation'].abs().idxmax()]
    print(":흰색_확인_표시: 시차별 상관관계:\n", lag_corr_df)
    print(f"\n:압정: 가장 높은 상관관계: lag +{int(best_row['Lag'])}, 상관계수 = {best_row['Correlation']:.4f}")
    # 시각화
    plt.figure(figsize=(8,5))
    plt.plot(lag_corr_df['Lag'], lag_corr_df['Correlation'], marker='o')
    plt.title("구글 트렌드 vs 미래 수익률 (Lag별 상관관계)")
    plt.xlabel("시차 (일)")
    plt.ylabel("상관계수")
    plt.grid(True)
    plt.show()
    return lag_corr_df, best_row

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
def lagged_sentiment_correlation(df_returns, sentiment_score, max_lag=7):
    """
    감성 점수(score)와 미래 수익률(Return)의 시차별 상관관계를 분석합니다.
    Parameters:
    - df_returns: '날짜', 'Return' 컬럼 포함 DataFrame
    - sentiment_score: '날짜', 'score' 컬럼 포함 DataFrame
    - max_lag: 최대 시차 일 수 (default = 7)
    Returns:
    - lag_corr_df: 시차별 상관계수 DataFrame
    - best_row: 가장 상관관계가 높은 시차 정보
    """
    # 날짜 포맷 통일
    df_returns['날짜'] = pd.to_datetime(df_returns['날짜'])
    sentiment_score['날짜'] = pd.to_datetime(sentiment_score['날짜'])
    results = []
    for lag in range(1, max_lag + 1):
        temp_returns = df_returns.copy()
        temp_returns['Return'] = temp_returns['Return'].shift(-lag)  # 미래 수익률
        # 날짜 기준 병합
        merged = pd.merge(temp_returns, sentiment_score, on='날짜', how='inner')
        merged_clean = merged.dropna(subset=['Return', 'score'])
        corr = merged_clean['Return'].corr(merged_clean['score'])
        results.append({'Lag': lag, 'Correlation': corr})
    lag_corr_df = pd.DataFrame(results)
    best_row = lag_corr_df.loc[lag_corr_df['Correlation'].abs().idxmax()]
    # 결과 출력
    print(":막대_차트: 시차별 상관관계 (감성점수 vs 미래 수익률):\n", lag_corr_df)
    print(f"\n:흰색_확인_표시: 최대 상관: lag +{best_row['Lag']}일, corr = {best_row['Correlation']:.4f}")
    # 시각화
    plt.figure(figsize=(8,5))
    plt.plot(lag_corr_df['Lag'], lag_corr_df['Correlation'], marker='o', color='tomato')
    plt.title("감성점수 vs 미래 수익률 (Lag별 상관관계)")
    plt.xlabel("시차 (일)")
    plt.ylabel("상관계수")
    plt.grid(True)
    plt.show()
    return lag_corr_df, best_row

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
def combined_signal_correlation(df_returns, google_trend, sentiment_score, max_lag=7):
    """
    (검색량 × 감성점수)와 미래 수익률(Return)의 시차 상관관계를 분석합니다.
    Parameters:
    - df_returns: '날짜', 'Return'
    - google_trend: '날짜', 'Trend'
    - sentiment_score: '날짜', 'score'
    - max_lag: 최대 시차 일수
    Returns:
    - lag_corr_df: 시차별 상관계수 DataFrame
    - best_row: 가장 높은 상관관계를 가진 시차 정보
    """
    # 날짜 통일
    df_returns['날짜'] = pd.to_datetime(df_returns['날짜'])
    google_trend['날짜'] = pd.to_datetime(google_trend['날짜'])
    sentiment_score['날짜'] = pd.to_datetime(sentiment_score['날짜'])
    # 1. Trend × score 계산
    merged_signal = pd.merge(google_trend, sentiment_score, on='날짜', how='inner')
    merged_signal['signal'] = merged_signal['Trend'] * merged_signal['score']
    results = []
    for lag in range(1, max_lag + 1):
        temp_returns = df_returns.copy()
        temp_returns['Return'] = temp_returns['Return'].shift(-lag)
        merged = pd.merge(temp_returns, merged_signal[['날짜', 'signal']], on='날짜', how='inner')
        merged_clean = merged.dropna(subset=['Return', 'signal'])
        corr = merged_clean['Return'].corr(merged_clean['signal'])
        results.append({'Lag': lag, 'Correlation': corr})
    lag_corr_df = pd.DataFrame(results)
    best_row = lag_corr_df.loc[lag_corr_df['Correlation'].abs().idxmax()]
    # 출력
    print("\n:막대_차트: 시차별 상관관계 (Trend × score vs 미래 수익률):\n", lag_corr_df)
    print(f"\n:흰색_확인_표시: 최대 상관: lag +{best_row['Lag']}일, corr = {best_row['Correlation']:.4f}")
    # 시각화
    plt.figure(figsize=(8,5))
    plt.plot(lag_corr_df['Lag'], lag_corr_df['Correlation'], marker='o', color='purple')
    plt.title("Trend × Score vs 미래 수익률 (Lag별 상관관계)")
    plt.xlabel("시차 (일)")
    plt.ylabel("상관계수")
    plt.grid(True)
    plt.show()
    return lag_corr_df, best_row